1. 문서의 내용을 읽는다.

2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림

3. 임베딩 -> 벡터 데이터베이스에 저장

4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색

5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달.

## DataSet

law.go.kr/법령/소득세법 -> download (doc)

In [ ]:
#%pip install --upgrade --quiet docx2txt langchain-community

### 1. 문서의 내용을 읽는다.

In [4]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader('./tax.docx')
document = loader.load()


In [ ]:
document

[Document(metadata={'source': './tax.docx'}, page_content='소득세법\n\n소득세법\n\n[시행 2025. 1. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n\n[본조신설 2009. 12. 31.]\n\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]\n\n\n\n제2조(납세의무

### 2. 문서를 쪼갠다.

In [ ]:
#%pip install -qU langchain-text-splitters

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # Recursive가 더 많은 조건을 적용해서 split 할 수 있다.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 하나의 chunk가 가질 수 있는 토큰 수
    chunk_overlap=200, # chunk가 겹치는 토큰 수 -> 우리가 원하는 내용을 가져올 확률을 높여주는 역할
)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list

### 3. 임베딩

In [8]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
#%pip install langchain-chroma

In [ ]:
from langchain_chroma import Chroma # Chroma : 벡터 DB

# 쪼개 놓은 document list를 embedding을 활용해서 저장한다.
#database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma_tax', persist_directory='./chroma')

# 데이터베이스를 생성한 다음에
database = Chroma(embedding=embedding, collection_name='chroma_tax', persist_directory='./chroma')

Chroma

InMemory 데이터여서 창을 닫으면 날아간다.

- chroma의 option에 from_documents에서 persist_directory='./chroma'를 추가한다.
- collection_name : table 이름 -> 추가한다.

chroma라는 폴더가 생기고 임베딩 결과를 chroma.sqlite3에 저장한다.

### 4. 유사도 검색

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# docs를 가져와서
retrieved_docs = database.similarity_search(query)

In [13]:
retrieved_docs

[Document(id='4fcf9964-9878-4485-85ba-9ea03d600be0', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

### 5. LLM에 질문

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

문서 없이 진행할 때는 그냥 질문을 던졌는데, 이번에는 prompt를 작성한다.
페르소나를 주면 좋다.

In [15]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [ ]:
# prompt를 짜서 llm에 invoke 해준다.
ai_message = llm.invoke(prompt)

In [ ]:
ai_message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 여러 단계의 계산이 필요합니다. 소득세 계산은 소득구간별 세율 및 공제를 고려해야 하기 때문에 복잡할 수 있습니다. 여기에서는 2023년을 기준으로 한 대략적인 절차를 설명하겠습니다:\n\n1. **소득구간에 따른 세율 적용**: 한국에서는 종합소득세율이 누진세율 구조를 가지고 있습니다. 연봉 5천만원은 다음과 같은 세율 구간에 해당됩니다:\n    - 1,200만원 이하: 6%\n    - 1,200만원 ~ 4,600만원: 15%\n    - 4,600만원 ~ 8,800만원: 24%\n    - 그 이상 구간은 필요 없음\n\n2. **과세표준 계산**: 총급여액에서 비과세 소득과 근로소득공제를 차감하여 과세표준을 계산합니다.\n    - **근로소득공제**: 이는 소득에 따라 달라지며, 대략적인 계산으로는 약 1,000만원 정도가 됩니다.\n    - 비과세 소득은 별도로 제공된 데이터에서 확인 가능하지 않으므로 기본 공제만 고려합니다.\n\n3. **세율 적용**: 각 소득구간에 따른 세율을 과세표준에 적용하여 세금을 계산합니다.\n    - 1,200만원 이하: 1,200만원 × 6% = 72만원\n    - 1,200만원 초과 4,600만원 이하: (4,600만원 - 1,200만원) × 15% = 510만원\n    - 4,600만원 초과 금액: (5천만원 - 4,600만원) × 24% = 96만원\n\n4. **총세금 계산**: 위의 계산들을 합하여 총소득세를 산출합니다.\n    - 총 소득세 = 72만원 + 510만원 + 96만원 = 678만원\n\n이는 단순화된 계산 예시이며, 실제로는 추가적인 공제 항목(예: 인적공제, 신용카드 소득공제 등)이 적용될 수 있습니다. 세무 상황에 따라 계산 결과가 달라질 수 있으므로, 정확한 세금 계산은 공인회계사나 세무사를 통해 확인하실 것을 권장합니다.'

**결과 text**

'직장인의 소득세 계산은 여러 요소에 의해 달라질 수 있습니다. 특히 연봉 5천만 원인 개인의 소득세를 계산할 때 다음의 요소들을 고려해야 합니다:\n\n1. **기본 소득 공제**: 근로소득 공제는 일정 금액 이하의 소득에 대해서는 고정 금액으로 공제되고, 그 이상에 대해서는 일정 비율로 공제됩니다. 연봉이 5천만 원일 경우, 근로소득 공제는 다음과 같이 계산될 수 있습니다.\n\n2. **세율 적용**: 한국 소득세는 누진세 구조로 되어 있어, 소득이 높아질수록 높은 세율이 적용됩니다. 2023년 기준으로 일반적인 소득세율은 다음과 같습니다:\n   - 1,200만 원 이하: 6%\n   - 1,200만 원 초과 ~ 4,600만 원 이하: 15%\n   - 4,600만 원 초과 ~ 8,800만 원 이하: 24%\n   - 그 외의 높은 구간에 대해서는 더 높은 세율이 적용됩니다.\n\n3. **중간 세율 계산**: 연봉 5천만 원은 기본적으로 4,600만 원까지는 15% 세율이 적용되고, 초과되는 400만 원에 대해서는 24%의 세율이 적용됩니다.\n\n4. **세금 공제 항목**: 근로소득세액공제나 다양한 개인 공제 항목(예: 부양가족 공제, 보험료 공제) 등이 있을 수 있으며, 이는 개인의 상황에 따라 다릅니다.\n\n구체적인 세금 계산은 근로소득 공제와 소득세율을 적용한 후 다양한 공제 항목을 반영하여 산출됩니다. 일반적으로는 세무사를 통해 세금을 정확히 계산하는 것이 좋습니다. 다만, 단순히 소득금액과 세율만으로 대략적인 계산을 시도할 수는 있습니다. 최종적인 세금 산출은 개인의 세무 상황에 따라 다를 수 있습니다.'

### Retrieval QA

In [ ]:
#%pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub

prompt = hub.pull('rlm/rag-prompt')

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# QA chain 생성

# langchain을 활용하면 llm 개발이 훨씬 편해진다.
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(), # langchain을 쓰면 다양한 벡터 데이터베이스에 대해 다 쓸 수 있다.
    chain_type_kwargs={
        'prompt': prompt
    }
)

In [ ]:
ai_message = qa_chain({"query": query})

In [23]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '그 질문에 대한 정보나 관련 내용이 제공된 문맥 안에 없어서, 구체적인 소득세 금액을 제공할 수 없습니다.'}